# Gerekli kütüphanelerin yüklenmesi

In [ ]:
!pip install transformers datasets wandb gradio pandas scikit-learn evaluate nltk rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=16867f3ad8d0428c10ccba980eb691960ddf02046bf96991b90067b9d9f3beb8
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


# Drive'a Bağlanma

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preprocessing

In [4]:
import pandas as pd
from datasets import Dataset

# CSV dosyasını yükleyin
df = pd.read_csv("/content/drive/MyDrive/medquad.csv")
df = df.dropna(subset=["question", "answer"])

# Seq2Seq formatına çevir
df_seq2seq = pd.DataFrame({
    "input_text": "question: " + df["question"],
    "target_text": df["answer"]
})

# HuggingFace Dataset'e çevir
dataset = Dataset.from_pandas(df_seq2seq)

## Dataset Bölünmesi

In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(df_seq2seq[["input_text", "target_text"]])
dataset = dataset.train_test_split(test_size=0.2)


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"  # Donanıma göre değiştirilebilir

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

## Veri Tokenizasyonu

In [ ]:
def preprocess(example):
    model_inputs = tokenizer(
        example["input_text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

    labels = tokenizer(
        example["target_text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/13125 [00:00<?, ? examples/s]

Map:   0%|          | 0/3282 [00:00<?, ? examples/s]

## Model Fine-Tune Edilmesi

In [7]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_total_limit=1,
)

## Model Eğitimi

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"], # Access the 'train' split of the dataset
    tokenizer=tokenizer,
)

<ipython-input-8-aa629953c9c3>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 33aliarslanzaa33 (33aliarslanzaa33-ali) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,17.025400
20,14.130600
30,13.219800
40,12.538500
50,10.364900
60,11.410800
70,8.962400
80,8.139800
90,7.632400
100,6.772100


TrainOutput(global_step=4923, training_loss=2.1690429107895377, metrics={'train_runtime': 1791.8736, 'train_samples_per_second': 21.974, 'train_steps_per_second': 2.747, 'total_flos': 3659719311360000.0, 'train_loss': 2.1690429107895377, 'epoch': 3.0})

# Fine-Tune Edilen Modelin Kaydedilmesi

In [ ]:
model.save_pretrained("flanT5Small_medquad_finetuned")
tokenizer.save_pretrained("flanT5Small_medquad_finetuned")

('flanT5Small_medquad_finetuned/tokenizer_config.json',
 'flanT5Small_medquad_finetuned/special_tokens_map.json',
 'flanT5Small_medquad_finetuned/spiece.model',
 'flanT5Small_medquad_finetuned/added_tokens.json',
 'flanT5Small_medquad_finetuned/tokenizer.json')

In [8]:
model_path = "/content/drive/MyDrive/flanT5Small_medquad_finetuned"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


#Chatbot Arayüzü: Gradio

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.9 MB/s eta 0:00:00


In [11]:
import gradio as gr

def answer_question(history, question):
    input_text = "question: " + question
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    outputs = model.generate(**inputs, max_new_tokens=128)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Sohbet geçmişine yeni çift ekle
    history = history + [(question, answer)]
    return history, ""

with gr.Blocks() as demo:
    gr.Markdown("## 🧠 MedQA Chatbot (Flan-T5)")

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Mesajınızı yazın", placeholder="Bir soru sorun...", lines=1)
    state = gr.State([])  # Sohbet geçmişi

    clear = gr.Button("Sohbeti Temizle")

    # Kullanıcı mesaj gönderdiğinde:
    msg.submit(fn=answer_question, inputs=[state, msg], outputs=[chatbot, msg])

    # Temizle butonuna basıldığında geçmiş ve input temizlenir
    clear.click(lambda: ([], ""), inputs=[], outputs=[chatbot, msg, state])

demo.launch()


<ipython-input-11-5694b79b5446>:16: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12a201ee3aea126b50.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#Değerlendirme: BLUE ve ROUGE

In [ ]:
from datasets import load_metric
import numpy as np


Error in callback <bound method _WandbInit._pre_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7a527613d810>> (for pre_run_cell):


FileNotFoundError: [Errno 2] No such file or directory: '/content/wandb/run-20250513_215545-yvx38tn1/logs/debug.log'

Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x7a527613d810>> (for post_run_cell):


FileNotFoundError: [Errno 2] No such file or directory: '/content/wandb/run-20250513_215545-yvx38tn1/logs/debug.log'

In [ ]:
import evaluate
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")


In [ ]:
# Küçük bir test seti alalım
# Access the 'test' split of the dataset
test_samples = dataset['test'].select(range(100))  # İlk 100 örnek üzerinden değerlendirme

predictions = []
references = []

for sample in test_samples:
    input_text = sample["input_text"]
    reference = sample["target_text"]

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model.generate(**inputs, max_new_tokens=128)

    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

    predictions.append(pred)
    references.append(reference)

In [ ]:
# BLEU
bleu_result = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print("BLEU:", bleu_result)

# ROUGE
rouge_result = rouge.compute(predictions=predictions, references=references)
print("ROUGE:", rouge_result)


BLEU: {'bleu': 0.0473885049362066, 'precisions': [0.3553113553113553, 0.16007416563658838, 0.12215269086357948, 0.10950570342205324], 'brevity_penalty': 0.28533528428331534, 'length_ratio': 0.4436379394399003, 'translation_length': 8190, 'reference_length': 18461}
ROUGE: {'rouge1': np.float64(0.22579366834876938), 'rouge2': np.float64(0.0963287908755748), 'rougeL': np.float64(0.19447658343479643), 'rougeLsum': np.float64(0.1954451560945777)}
